In [236]:
import os
import openai
import sys
import json
sys.path.append('../..')

In [237]:
openai.api_key = os.environ['OPENAI_API_KEY']

In [238]:
# from langchain.prompts.chat import (
#     ChatPromptTemplate,
#     HumanMessagePromptTemplate,
#     SystemMessagePromptTemplate,
# )
# from langchain.schema import HumanMessage, SystemMessage
# from langchain_openai import ChatOpenAI
# chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [239]:
# messages = [
#     SystemMessage(
#         content="You are a helpful assistant that translates English to French."
#     ),
#     HumanMessage(
#         content="Translate this sentence from English to Pirate. I love programming."
#     ),
# ]
# chat.invoke(messages)

In [240]:
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import ChatOpenAI

# prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
# model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1)
# output_parser = StrOutputParser()

# chain = prompt | model | output_parser

# chain.invoke({"topic": "ice cream"})

In [248]:
import random
def design_logo_concepts(topic: str):
    model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    template = """Answer yes if the user will need a logo, otherwise, answer no. Here is the user prompt

    Prompt = {prompt}"""
    
    prompt = ChatPromptTemplate.from_template(template)
    output_parser = StrOutputParser()
    inner_chain = prompt | model | output_parser
    if 'Yes' in inner_chain.invoke({"prompt" : topic}):
        index = random.randint(1,4)
        s3_path = f's3://designs/logo_{index}.jpg'
        return f'{{ component: \"Image\", props: {{ url: {s3_path}, }} }}'
    return 'NONE'

In [249]:
from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnableLambda
from langchain.schema.runnable import RunnableMap
from langchain_openai import ChatOpenAI

template = """If KEY is NONE, then as a smart agent, you must kindly respond to the prompt given below. 
prompt: {prompt}

Otherwise, simply answer with the given KEY

KEY: {response}

"""

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
output_parser = StrOutputParser()

chain = RunnableMap({
    "response": lambda x : design_logo_concepts(x["prompt"]),
    "prompt": lambda x : x["prompt"]
}) | prompt | model | output_parser

In [250]:
def interact_with_model(input):
    output = chain.invoke({"prompt": input})
    try:
        return json.loads(output)
    except:
        return output

In [251]:
interact_with_model("hi")

'Hi! How can I assist you today?'

In [252]:
interact_with_model("I think ice cream is great")

'I think ice cream is great.'

In [253]:
interact_with_model("I would like an ice cream truck logo")

{'component': 'Image', 'props': {'url': 's3://designs/logo_3.jpg'}}

In [254]:
interact_with_model("bye bye")

'Goodbye!'

In [255]:
interact_with_model("NOOOO, I need another logo, perhaps a logo for my UI task")

{'component': 'Image', 'props': {'url': 's3://designs/logo_4.jpg'}}